# Create boxdata
* Input: label data
* Output: numpy array CT iamge and label (pancreas, lesion)

In [1]:
import time
import os, glob, ntpath
import logging
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
import pydicom as dicom
import nrrd
import matplotlib.pyplot as plt
from datetime import datetime as ddt

In [2]:
from create_boxdata import create_boxdata

/home/u/sonic81518/.conda/envs/tinghui/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
border = np.array([0, 0, 0])
box_save_path = '/home/d/pancreas/test/'

In [4]:
sortkey = lambda k: ntpath.basename(k).replace('.dcm', '').split('-')[-1]
sortkey_PC = lambda k: int(ntpath.basename(k).replace('.dcm', '').split('I')[-1])

# Usage
To view one simple case, just simply run `create_box_data`. It will return the numpy array of image and labels. <br>
For NP and PT, please input **sortkey**; for PC, please input **sortkey_PC**.

In [7]:
image, label = create_boxdata('/home/d/pancreas/label_data/000027/PC67/', sortkey_PC)

In [8]:
print('Image shape:', image.shape)
print('Number of content in label:', len(label))
print('Total label name:')
for item in label:
    print(' ', item[0], ':', item[1].shape)

Image shape: (110, 58, 14)
Number of content in label: 3
Total label name:
  pancreas : (110, 58, 14)
  lesion : (110, 58, 14)
  pduct : (110, 58, 14)


# Create box data and save as numpy array

In [5]:
st_tol = time.time()
cnt = 0
for tumorpath in tqdm(glob.glob('/home/d/pancreas/label_data/*/PT*/*nrrd')):
    create_boxdata(tumorpath.replace('/label.nrrd','/'), sortkey, savefile=True, box_save_path=box_save_path)
    cnt += 1
print('Done create {} box data in {} seconds'.format(cnt, time.time()-st_tol))

HBox(children=(IntProgress(value=0, max=44), HTML(value='')))

/home/u/sonic81518/.conda/envs/tinghui/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)



Done create 44 box data in 29.50455069541931 seconds


In [6]:
st_tol = time.time()
cnt = 0
for tumorpath in tqdm(glob.glob('/home/d/pancreas/label_data/*/NP*/*nrrd')):
    create_boxdata(tumorpath.replace('/label.nrrd','/'), sortkey, savefile=True, box_save_path=box_save_path)
    cnt += 1
print('Done create {} box data in {} seconds'.format(cnt, time.time()-st_tol))

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))


Done create 10 box data in 4.836451530456543 seconds


In [7]:
st_tol = time.time()
cnt = 0
for tumorpath in tqdm(glob.glob('/home/d/pancreas/label_data/*/PC*/*nrrd')):
    try:
        create_box_data(tumorpath.replace('/label.nrrd','/'), sortkey_PC, savefile=True, box_save_path=box_save_path)
        cnt += 1
    except:
        print(tumorpath)
print('Done create {} box data in {} seconds'.format(cnt, time.time()-st_tol))

HBox(children=(IntProgress(value=0, max=99), HTML(value='')))

/home/u/sonic81518/.conda/envs/tinghui/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)



Done create 99 box data in 15.93248462677002 seconds
